In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
# import required packages
import requests
import urllib.request as urllib2
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np
import re
import os
import glob

import datetime
from datetime import date
from datetime import datetime

In [ ]:
team_names={"Mumbai Indians": "MI",
            "Mum Indians":"MI",
            "Chennai Super Kings": "CSK",
            "Super Kings": "CSK",
            "Kolkata Knight Riders":"KKR",
            "Royal Challengers Bangalore":"RCB",
            "Kings XI Punjab":"KXIP",
            "Kings XI":"KXIP",
            "Rajasthan Royals":"RR",
            "Royals":"RR",
            "Delhi Capitals":"DC",
            "Capitals":"DC",
            "Delhi Daredevils":"DC",
            "Sunrisers Hyderabad":"SRH",
            "Sunrisers":"SRH",
            "Deccan Chargers":"SRH",
            "Gujarat Lions":"GL",
            "Pune Warriors":"RPS",
            "Rising Pune Supergiant":"RPS",
            "Kochi Tuskers Kerala":"KTK",
            "Rising Pune Supergiants":"RPS",
           }

In [ ]:
directory = '/content/drive/My Drive/IPL Data/webpage/'
subdirs = [os.path.join(directory, subdir) for subdir in os.listdir(directory) if os.path.isdir(os.path.join(directory,subdir))]

all_htmls = []
for subdir in subdirs:
  innings = []
  for inning in glob.glob(subdir+"/*inning*"):
   innings.append(inning)
  for scorecard in glob.glob(subdir+"/*Scorecard*"):
    innings.append([scorecard])
  for superover in glob.glob(subdir+"/*superover*"):
    innings.append([superover])
    
  all_htmls.append(innings)

In [ ]:
def nonstrike(dataset):
	subset = dataset
	if len(subset.batsman.unique().tolist()) == 1:
		for i in range(len(subset)):
			subset.non_striker[i] = ""
		return subset
	else:
		for i in range(len(subset)):
			non_striker = [b for b in subset.batsman.unique().tolist() if subset.batsman[i] not in b]
			subset.non_striker[i] = non_striker[0]
		return subset

In [ ]:
%%capture
#Scraping Match Summary data
#opening html file with BeautifulSoup
matches_path = '/content/drive/My Drive/IPL Data/webpage/Indian Premier League, 2020_21 Cricket Team Records & Stats _ ESPNcricinfo.com.html'
with open(matches_path) as fp:
	matches_soup = BeautifulSoup(fp, "html5lib")
	
#create an empty dataframe
matches_df = pd.DataFrame(columns=['team1', 'team2', 'winner', 'margin', 'city', 'date', 'id'])
	
#scraping the match summary data
table_body=matches_soup.find_all('tbody')
for i, table in enumerate(table_body[0:4:2]):
	rows = table.find_all('tr')
	for row in rows[::1]:
		
		id = str(row.find_all('a')[-1])
		id = id.split('.html')[0].split('/')[-1]
	
		cols=row.find_all('td')
		cols=[x.text.strip() for x in cols][:-1]
		cols.append(id)
	
		matches_df.loc[len(matches_df)] = cols

matches_df.date = pd.to_datetime(matches_df.date)
	
season = int(str(matches_df.date[0])[:4])
matches_df['season'] = season
	
matches_df['result'] = 'normal'
for i in range(len(matches_df)):
	if 'tied' in matches_df.winner[i]:
		matches_df['result'][i] = 'tie'

matches_df['win_by_runs'] = 0
matches_df['win_by_wickets'] = 0

for i in range(len(matches_df)):

	if 'wickets' in matches_df.margin[i]:
		matches_df['win_by_wickets'][i] = int(matches_df.margin[i].split(' ')[0])

	if 'run' in matches_df.margin[i]:
		matches_df['win_by_runs'][i] = int(matches_df.margin[i].split(' ')[0])

del  matches_df['margin']
matches_df = matches_df[['id', 'season', 'city', 'date', 'team1', 'team2', 'result', 'winner', 'win_by_runs', 'win_by_wickets']]
matches_df['team1'].replace(team_names, inplace=True)
matches_df['team2'].replace(team_names, inplace=True)

matches_df.insert(loc=6, column='toss_winner', value="")
matches_df.insert(loc=7, column='toss_decision', value="")
matches_df.insert(loc=9, column='dl_applied', value=0)
matches_df.insert(loc=13, column='player_of_match', value="")
matches_df.insert(loc=14, column='venue', value="")
matches_df.insert(loc=15, column='umpire1', value="")
matches_df.insert(loc=16, column='umpire2', value="")
matches_df.insert(loc=17, column='umpire3', value="")

In [ ]:
%%capture

deliveries_wip = pd.DataFrame()
matches_wip = pd.DataFrame()

for htmls in all_htmls:
	commentary_path1 = htmls[0]
	commentary_path2 = htmls[1]
	scorecard_path = htmls[2][0]
  
	commentary_paths = []
	commentary_paths.append(commentary_path1)
	commentary_paths.append(commentary_path2)
  
	if len(htmls) == 4:
		commentary_path3 = htmls[3][0]
		commentary_paths.append(commentary_path3)

	commentary = pd.DataFrame()
	summary = pd.DataFrame()
	for commentary_path in commentary_paths:
		#Load the correct html file
		# opening html file with BeautifulSoup
		with open(commentary_path) as fp:
			innings_soup = BeautifulSoup(fp, "html5lib")
		
		alldata = []
		for div in innings_soup.findAll('div', {'class': 'd-flex match-comment-padder align-items-center'}):
			alldata.append(div.text)
		
		supersets = []
		if len(alldata) < 20:
			alldata.reverse()
			super_index = alldata.index([item for item in alldata if '0.1' in item][1])
			supersets.append(alldata[:super_index])
			supersets.append(alldata[super_index:])
		else:
			supersets.append(alldata)
		
		
		for sets in supersets:
			alldata = sets
		
			ball_df = pd.DataFrame()
			for i in range(len(alldata)):
				over = alldata[i].split('.')[0]
				ball = alldata[i].split('.')[1][0]
				run =  alldata[i][len(over)+2]
				desc = alldata[i][len(over)+3:].split(',')[0]
				bowler,batsman = desc.split(' to ')
				if ' Patel' in bowler:
					bowler = re.findall('[A-Z][^A-Z]*', bowler)[0]+re.findall('[A-Z][^A-Z]*', bowler)[1]+" "+re.findall('[A-Z][^A-Z]*', bowler)[2]
				else:
					bowler = re.findall('[A-Z][^A-Z]*', bowler)[0]
				bastman = batsman[:-1]
				over = int(over)+1
				if (' retired hurt ' in alldata[i]) and (run != 'w'):
					injury = 1
				else:
					injury = 0
				row	= pd.DataFrame([over,ball,run,injury,batsman,bowler]).T
				ball_df = ball_df.append(row)
			
			ball_df.columns = ['over','ball','run','injury','batsman','bowler']
			ball_df['over'] = ball_df['over'].astype(int)
			ball_df['ball'] = ball_df['ball'].astype(int)
			ball_df = ball_df.sort_values(["over", "ball"], ascending = (True, True)).reset_index(drop=True)
			
			indexes = []
			for i in range(len(ball_df)):
				if ball_df.run[i] =='w' or (ball_df.injury[i] ==1):
					indexes.append(i)
			indexes.append(999)
			
			ball_df['non_striker'] = ""
			allsets = pd.DataFrame()
			for i in range(len(indexes)):
			
				if i == 0:
					start = 0
					end = indexes[i]
					subset = ball_df[start:end].reset_index(drop=True)
					if len(subset) < 1:
						continue
					else:
						subset_first = nonstrike(subset)
						allsets = allsets.append(subset_first)
			
				elif i == len(indexes)-1:
					start = indexes[i-1]+1
					subset = ball_df[start:].reset_index(drop=True)
					if len(subset) < 1:
						continue
					else:
						subset_last = nonstrike(subset)
						allsets = allsets.append(subset_last)
			
				elif indexes[i]-1 == indexes[i-1]:
					continue
			
				else:
					start = indexes[i-1]+1
					end = indexes[i]
					subset = ball_df[start:end].reset_index(drop=True)
					if len(subset) < 1:
						continue
					else:
						subset_interim = nonstrike(subset)
						allsets = allsets.append(subset_interim)
			
			for i in indexes:
				if i == 999:
					continue
				allsets = allsets.append(ball_df[i:i+1])
			
			allsets = allsets.sort_values(["over", "ball"], ascending = (True, True)).reset_index(drop=True)
			
			for i in indexes:
				if (i == 999) or (i == 0) or (allsets.run[i-1] == 'w') or (allsets.injury[i-1] == 1):
					continue
				else:
					player1 = allsets.batsman[i-1]
					player2 = allsets.non_striker[i-1]
					if allsets.batsman[i] == player1:
						allsets.non_striker[i] = player2
					else:
						allsets.non_striker[i] = player1
			
			for i in range(len(allsets)):
				if i == 0:
					continue
				elif allsets.non_striker[i] == "":
					allsets.non_striker[i] = allsets.non_striker[i-1]
			
			for i in range(len(allsets)):
				if allsets.batsman[i] == allsets.non_striker[i]:
					allsets.non_striker[i] = ""
			
			for i in range(len(allsets)):
				if allsets.non_striker[i] == "":
					player_list = []
					for n in range(i+1,len(allsets)):
						if allsets.non_striker[n] == "":
							continue
						else:
							player = allsets.non_striker[n]
							player_list.append(player)
					if len(player_list) != 0:
						allsets.non_striker[i] = player_list[0]
			
			# for score on each ball
			ball_score =[]
			for div in innings_soup.findAll('div', {'class': 'match-comment-run'}):
				ball_score.append(div.text)
			
			ball_score.reverse()
			ball_df = allsets
			del ball_df['run']
			del ball_df['injury']
			if (len(alldata) < 20) and (supersets.index(alldata) == 0):
				ball_df.insert(loc=2, column='score', value=ball_score[:super_index])
			elif (len(alldata) < 20) and (supersets.index(alldata) == 1):
				ball_df.insert(loc=2, column='score', value=ball_score[super_index:])
			else:
				ball_df.insert(loc=2, column='score', value=ball_score)
					
			#create	placeholder columns as per the given format
			ball_df['is_super_over'] = 0
			ball_df['wide_runs'] = 0
			ball_df['bye_runs'] = 0
			ball_df['legbye_runs'] = 0
			ball_df['noball_runs'] = 0
			ball_df['penalty_runs'] = 0
			ball_df['batsman_runs'] = 0
			ball_df['extra_runs'] = 0
			ball_df['total_runs'] = 0
			ball_df['player_dismissed'] = ""
			ball_df['dismissal_kind'] = ""
			ball_df['fielder'] = ""
			
			#super over treatment
			if len(alldata) < 20:
				ball_df['is_super_over'] = 1			
			
			#dot balls treatment
			for i in range(len(ball_df)):
				if ball_df['score'][i] == '•':
					ball_df['score'][i] = '0'
			
			#wide,leg bye and no balls treatment and dismissal
			for i in range(len(ball_df)):
				if ball_df['score'][i] == 'w':
					ball_df['player_dismissed'][i] = 1
				elif len(ball_df['score'][i]) == 1:
					continue
				elif  ball_df['score'][i][1] == 'w':
					ball_df['wide_runs'][i] = ball_df['score'][i][0]
					if ball_df['score'][i][0] == '0':
						ball_df['wide_runs'][i] = 1
				elif ball_df['score'][i][1] == 'n':
					ball_df['noball_runs'][i] = ball_df['score'][i][0] 
					if ball_df['score'][i][0] == '0':
						ball_df['noball_runs'][i] = 1
				elif ball_df['score'][i][1] == 'l':
					ball_df['legbye_runs'][i] = ball_df['score'][i][0]
					if ball_df['score'][i][0] == '0':
						ball_df['legbye_runs'][i] = 1	
				elif ball_df['score'][i][1] == 'b':
					ball_df['bye_runs'][i] = ball_df['score'][i][0]
					if ball_df['score'][i][0] == '0':
						ball_df['bye_runs'][i] = 1	
			
			#Columns involving runs and extras
			for i in range(len(ball_df)):
				if ball_df['score'][i] == 'w':
					ball_df['score'][i] = '0'
				ball_df['total_runs'][i] = ball_df['score'][i][0]
			
			ball_df['extra_runs'] = ball_df['wide_runs'] + ball_df['bye_runs'] + ball_df['legbye_runs'] + ball_df['noball_runs'] + 	ball_df['penalty_runs']	
			ball_df['total_runs'] = ball_df['total_runs']. astype(str). astype(int)
			ball_df['extra_runs'] = ball_df['extra_runs']. astype(str). astype(int)
			ball_df['batsman_runs'] = ball_df['total_runs'] - ball_df['extra_runs']
			for i in range(len(ball_df)):
				if ball_df['batsman_runs'][i] < 0:
					ball_df['batsman_runs'][i] = 0	
			ball_df['total_runs'] = ball_df['batsman_runs'] + ball_df['extra_runs']
			del ball_df['score']
			
			#Ball number adjustment for extras
			for i in range(len(ball_df)):
				if (ball_df['wide_runs'][i] != 0) or (ball_df['noball_runs'][i] != 0):
					over = ball_df['over'][i]
					ball = ball_df['ball'][i]
				
					indices = ball_df[ball_df.over == over][ball_df.ball >= ball].index
					for index in indices:
						if index == min(indices):
							continue
						else:
							ball_df.ball[index] = ball_df.ball[index]+1
			
			# create an empty dataframe for capturing dismissal details
			dismissal_df = pd.DataFrame(columns=['batsman', 'bowler', 'dismissal_kind', 'fielder'])
			
			ball_out = list()
			# for dismissal on each ball
			for div in innings_soup.findAll('div', {'class': 'match-comment-wicket match-comment-wicket-no-icon'}):
				ball_out.append(div.text)
			
			#Creating dismissal dataframe
			dis_batsman=[]
			dis_bowler=[]
			dismissal_kind=[]
			dis_fielder=[]
			
			for item in ball_out:
		
				if (' obstructing the field ' in item):
					batsman = item.split(' obstructing the field ')[0]
					fielder = ""
					bowler = ""
					kind = 'obstructing the field'
				
				if (' hit wicket ' in item):
					batsman,bowler = item.split(' hit wicket b ')
					fielder = ""
					if len(bowler.split(' ')[0])<3:
						bowler = bowler.split(' ')[0]+" "+bowler.split(' ')[1]
					else:
						bowler = bowler.split(' ')[0]
					kind = 'hit wicket'
					
				if (' run out ' in item):
					batsman,fielder = item.split(' run out ')
					fielder = fielder.split('/')[0][1:]
					if 'sub' in fielder:
						fielder = re.sub('[^a-zA-Z]+', '', fielder.split('sub')[1])
					elif ')' in fielder:
						fielder = fielder.split(')')[0]
					bowler = ""
					kind = 'run out'
			
				if (' st ' in item):
					batsman,fielder = item.split(' st ')
					fielder, bowler = fielder.split(' b ')
					if 'sub' in fielder:
						fielder = re.sub('[^a-zA-Z]+', '', fielder.split('sub')[1])
					if len(bowler.split(' ')[0])<4:
						bowler = bowler.split(' ')[0]+" "+bowler.split(' ')[1]
					else:
						bowler = bowler.split(' ')[0]
					kind = 'stumped'
			
				elif ('  b ' in item) and (' c ' not in item):
					batsman,bowler = item.split('  b ')
					fielder = ""
					if len(bowler.split(' ')[0])<3:
						bowler = bowler.split(' ')[0]+" "+bowler.split(' ')[1]
					else:
						bowler = bowler.split(' ')[0]
					kind = 'bowled'
			
				elif ' c & b ' in item:
					batsman,bowler = item.split(' c & b ')
					if len(bowler.split(' ')[0])<4:
						bowler = bowler.split(' ')[0]+" "+bowler.split(' ')[1]
					else:
						bowler = bowler.split(' ')[0]
					fielder = bowler
					kind = 'caught and bowled'
				
				elif ' lbw b ' in item:
					batsman,bowler = item.split(' lbw b ')
					fielder = ""
					if len(bowler.split(' ')[0])<3:
						bowler = bowler.split(' ')[0]+" "+bowler.split(' ')[1]
					else:
						bowler = bowler.split(' ')[0]
					kind = 'lbw'
				
				elif ' c ' in item:
					batsman,fielder = item.split(' c ')
					fielder,bowler = fielder.split(' b ')
					if 'sub' in fielder:
						fielder = re.sub('[^a-zA-Z]+', '', fielder.split('sub')[1])
					if len(bowler.split(' ')[0])<4:
						bowler = bowler.split(' ')[0]+" "+bowler.split(' ')[1]
					else:
						bowler = bowler.split(' ')[0]
					kind = 'caught'
				
				dis_batsman.append(batsman)
				dis_fielder.append(fielder)
				dis_bowler.append(bowler)
				dismissal_kind.append(kind)
			
			# save lists in respective dismissal dataframe columns
			dismissal_df.batsman = dis_batsman
			dismissal_df.bowler = dis_bowler
			dismissal_df.dismissal_kind = dismissal_kind
			dismissal_df.fielder = dis_fielder
			
			for i in range(len(dismissal_df)):
				if '†' in dismissal_df.fielder[i]:
					word = dismissal_df.fielder[i]
					new_word = ''.join(word.split('†'))
					dismissal_df.fielder[i] = new_word
			
			#incorporating dismissal details in the main dataframe
			for i in range(len(ball_df)):
				if ball_df['player_dismissed'][i] != "":
					player1 = ball_df['batsman'][i]
					player2 = ball_df['non_striker'][i]
					if len(ball_df['non_striker'][i]) == 0:
						player2 = 'null'
				
					for n in range(len(dismissal_df)):
						if (player1 in dismissal_df['batsman'][n]) or (player2 in dismissal_df['batsman'][n]) :
							displayer = dismissal_df['batsman'][n]
							fielder = dismissal_df['fielder'][n]
							kind = dismissal_df['dismissal_kind'][n]
							ball_df['player_dismissed'][i] = displayer
							ball_df['fielder'][i] = fielder
							ball_df['dismissal_kind'][i] = kind    
			
			#Match ID
			meta_lines=[]
			for line in innings_soup.find_all('meta'):
				meta_lines.append(str(line))
			
			for line in meta_lines:
				if ('https' in line) & ('vs' in line) & ('description"/>' not in line):
					match_id = line.split('commentary/')[1].split('/')[0]
			
			#columns related to team names and match innings 
			team = []
			for div in innings_soup.findAll('span', {'class': 'no-wrap'}):
				team.append(div.text)
			teamname = team[0].split(':')[0]
			for	i in range(len(team)):
				if 'RRR' in team[i]:
					inning = 2
				else:
					inning = 1
			
			if (len(alldata) < 20) and (supersets.index(alldata) == 0):
				inning = 1
				teamname = team[1].split(':')[0]
			elif (len(alldata) < 20) and (supersets.index(alldata) == 1):
				inning = 2
				teamname = team[0].split(':')[0]

			if teamname in matches_df[matches_df.id == match_id]['team1'].reset_index(drop=True)[0]:
				bowling_team = matches_df[matches_df.id == match_id]['team2'].reset_index(drop=True)[0]
			else:
				bowling_team = matches_df[matches_df.id == match_id]['team1'].reset_index(drop=True)[0]
			
			ball_df['batting_team'] = teamname
			ball_df.insert(loc=0, column='inning', value = inning)
			ball_df.insert(loc=0, column='match_id', value= match_id)
			ball_df.insert(loc=3, column='bowling_team', value= bowling_team)
			
			#Player and Teams
			deliveries = pd.read_csv("/content/drive/My Drive/IPL Data/deliveries.csv")
			deliveries['batting_team'].replace(team_names, inplace=True)
			deliveries['bowling_team'].replace(team_names, inplace=True)
			deliveries = deliveries.fillna("")
			
			
			teamplayers_bat = {}
			for team in deliveries.batting_team.unique():
				team_df = deliveries[deliveries.batting_team == team]
				for batsman in team_df.batsman.unique():
					if team in teamplayers_bat.keys():
						teamplayers_bat[team].append(batsman)
					else:
						teamplayers_bat[team] = [batsman]
			
			teamplayers_bowl = {}
			for team in deliveries.bowling_team.unique():
				team_df = deliveries[deliveries.bowling_team == team]
				for bowler in team_df.bowler.unique():
					if team in teamplayers_bowl.keys():
						teamplayers_bowl[team].append(bowler)
					else:
						teamplayers_bowl[team] = [bowler]
			
			for key in teamplayers_bowl: 
				if key in teamplayers_bat: 
					teamplayers_bowl[key].extend(teamplayers_bat[key])
			
			for key in teamplayers_bowl.keys():
				unique_list = []
				for x in teamplayers_bowl[key]:
					if x not in unique_list:
						unique_list.append(x)
				teamplayers_bowl[key] = unique_list
			
			teamplayers = teamplayers_bowl
			
			if 'S Curran' in teamplayers['RR']:
				teamplayers['RR'].remove('S Curran')
			
			if 'T Curran' not in teamplayers['RR']:
				teamplayers['RR'].append('T Curran')
		
			if 'Garg' in teamplayers['SRH']:
				teamplayers['SRH'].remove('Garg')
				teamplayers['SRH'].append('Priyam Garg')
				
			if 'Nortje' in teamplayers['DC']:
				teamplayers['DC'].remove('Nortje')
				teamplayers['DC'].append('Anrich Nortje')
			
			players=[]
			for l in teamplayers.values():
				for i in l:
					players.append(i)
			
			unique_list = []
			for x in players:
				if x not in unique_list:
					unique_list.append(x)
			players	= unique_list
			
			#Renaming players in main dataframe
			for i in range(len(ball_df)):
				batsman =ball_df.batsman[i]
				for player in teamplayers[teamname]:
					if batsman in player:
						ball_df.batsman[i] = player
			
			for i in range(len(ball_df)):
				nonstriker =ball_df.non_striker[i]
				if nonstriker == "":
					continue
				else:
					for player in teamplayers[teamname]:
						if nonstriker in player:
							ball_df.non_striker[i] = player
			
			for i in range(len(ball_df)):
				bowler =ball_df.bowler[i]
				for player in teamplayers[bowling_team]:
					if bowler in player:
						ball_df.bowler[i] = player
			
			for i in range(len(ball_df)):
				if ball_df.player_dismissed[i] == "":
					continue
				dismissed =ball_df.player_dismissed[i]
				for	player in teamplayers[teamname]:
					if dismissed in player:
						ball_df.player_dismissed[i] = player
			
			for i in range(len(ball_df)):
				if ball_df.fielder[i] == "":
					continue
				fielder =ball_df.fielder[i]
				for	player in teamplayers[bowling_team]:
					if fielder in player:
						ball_df.fielder[i] = player
			
			for i in range(len(ball_df)):
				batsman =ball_df.batsman[i]
				for player in players:
					if batsman in player:
						ball_df.batsman[i] = player
			
			for i in range(len(ball_df)):
				nonstriker =ball_df.non_striker[i]
				if nonstriker == "":
					continue
				else:
					for player in players:
						if nonstriker in player:
							ball_df.non_striker[i] = player
			
			for i in range(len(ball_df)):
				bowler =ball_df.bowler[i]
				for player in players:
					if bowler in player:
						ball_df.bowler[i] = player
			
			for i in range(len(ball_df)):
				if ball_df.player_dismissed[i] == "":
					continue
				dismissed =ball_df.player_dismissed[i]
				for	player in players:
					if dismissed in player:
						ball_df.player_dismissed[i] = player
			
			for i in range(len(ball_df)):
				if ball_df.fielder[i] == "":
					continue
				fielder =ball_df.fielder[i]
				for	player in players:
					if fielder in player:
						ball_df.fielder[i] = player
			
			#Finally sorting the ball by ball dataframe
			ball_df = ball_df.sort_values(["over", "ball"], ascending = (True, True)).reset_index(drop=True)
			ball_df = ball_df[['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball', 
							'batsman', 'non_striker', 'bowler','is_super_over', 'wide_runs',
							'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
							'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
							'dismissal_kind', 'fielder']]
			
			#Player of the Match and Toss
			# opening html file with BeautifulSoup
			with open(scorecard_path) as fp:
				scorecard_soup = BeautifulSoup(fp, "html5lib")
			
			#scraping the match summary data
			table_body=scorecard_soup.find_all('tbody')
			for line in table_body:
				if 'elected to' in str(line):
					table = line
			rows = table.find_all('tr')
			
			venue = rows[0].find_all('td')
			venue = [x.text.strip() for x in venue][0]
			
			toss = rows[1].find_all('td')
			toss = [x.text.strip() for x in toss]
			
			motm = rows[3].find_all('td')
			motm = [x.text.strip() for x in motm]
			
			point = rows[-1].find_all('td')
			point = [x.text.strip() for x in point]
					
			umpire_rows=[]
			for row in rows:
				if 'Umpire' in row.text.strip():
					umpire_rows.append(row)
		
			umpires = umpire_rows[0].find_all('td')
			umpires = [x.text.strip() for x in umpires][1]
			umpire1 = umpires.split(" ")[0]+" "+re.findall('[A-Z][^A-Z]*', umpires.split(" ")[1])[0]
			umpire2 = re.findall('[A-Z][^A-Z]*', umpires.split(" ")[1])[1]+" "+umpires.split(" ")[-1]
	
			third_umpire = umpire_rows[1].find_all('td')
			third_umpire = [x.text.strip() for x in third_umpire][1]
			
			#Toss data
			toss_winner = toss[1].split(' , ')[0]
			if 'bat' in toss[1].split(' , ')[1]:
				toss_decision = 'bat'
			else:
				toss_decision = 'field'
			
			#player of the match
			player_of_match = motm[1]
			matchplayer = player_of_match.split(" ")[-1]
			
			for player in ball_df.batsman.unique():
				if matchplayer in player:
					matchplayer = player
			
			for player in ball_df.bowler.unique():
				if matchplayer in player:
					matchplayer = player
			
			#winner when result is tied
			if int(point[1].split(',')[0][-1]) == 2:
				winner = point[1].split(',')[0][:-2]
			else:
				winner = point[1].split(',')[1][1:-2]
		
			index = matches_df[matches_df.id == match_id].index[0]
			matches_df['toss_winner'][index] = toss_winner
			matches_df['toss_winner'].replace(team_names, inplace=True)
			matches_df['toss_decision'][index] = toss_decision
			if matches_df['winner'][index] == 'tied':
				matches_df['winner'][index] = winner
			matches_df['winner'].replace(team_names, inplace=True)
			matches_df['player_of_match'][index] = matchplayer
			matches_df['venue'][index] = venue
			matches_df['umpire1'][index] = umpire1
			matches_df['umpire2'][index] = umpire2
			matches_df['umpire3'][index] = third_umpire
			
			commentary = commentary.append(ball_df)
			if len(alldata) > 20:
				commentary = commentary.sort_values(["inning","over", "ball"], ascending = (True,True, True)).reset_index(drop=True)
			
			if ' ' not in matchplayer:
				print('check player of the match from the other matches_wip file.')
			elif ' ' in matchplayer:
				summary = matches_df
				current_match = summary[summary.id == match_id].reset_index(drop=True)
				current_match.date = current_match.date.astype(str)
		
	deliveries_wip = deliveries_wip.append(commentary)
	matches_wip = matches_wip.append(current_match).reset_index(drop=True)
		
#deliveries_wip.to_csv("/content/drive/My Drive/IPL Data/webpage/Deliveries IPL 2020.csv",index=False)
#matches_wip.to_csv("/content/drive/My Drive/IPL Data/webpage/Matches IPL 2020.csv",index=False)

In [ ]:
deliveries_wip[deliveries_wip.is_super_over == 1]

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
494,1216493,1,KXIP,DC,1,1,KL Rahul,,K Rabada,1,0,0,0,0,0,2,0,2,,,
495,1216493,1,KXIP,DC,1,2,KL Rahul,,K Rabada,1,0,0,0,0,0,0,0,0,KL Rahul,caught,HV Patel
496,1216493,1,KXIP,DC,1,3,N Pooran,,K Rabada,1,0,0,0,0,0,0,0,0,N Pooran,bowled,
497,1216493,2,DC,KXIP,1,1,RR Pant,SS Iyer,Mohammed Shami,1,0,0,0,0,0,0,0,0,,,
498,1216493,2,DC,KXIP,1,2,RR Pant,SS Iyer,Mohammed Shami,1,1,0,0,0,0,0,1,1,,,
499,1216493,2,DC,KXIP,1,3,RR Pant,SS Iyer,Mohammed Shami,1,0,0,0,0,0,2,0,2,,,
2477,1216547,1,MI,RCB,1,1,KA Pollard,HH Pandya,N Saini,1,0,0,0,0,0,1,0,1,,,
2478,1216547,1,MI,RCB,1,2,HH Pandya,KA Pollard,N Saini,1,0,0,0,0,0,1,0,1,,,
2479,1216547,1,MI,RCB,1,3,KA Pollard,HH Pandya,N Saini,1,0,0,0,0,0,0,0,0,,,
2480,1216547,1,MI,RCB,1,4,KA Pollard,HH Pandya,N Saini,1,0,0,0,0,0,4,0,4,,,


In [ ]:
%%capture
#manually correcting match:1216493 
deliveries_wip = deliveries_wip.reset_index(drop=True)
for i in range(len(deliveries_wip)):
  if (deliveries_wip.match_id[i] == '1216493') and (deliveries_wip.is_super_over[i] == 1) and (deliveries_wip.batting_team[i] == 'DC'):
    deliveries_wip.non_striker[i] = 'SS Iyer'
    if  deliveries_wip.ball[i] == 4:
      deliveries_wip.ball[i] = 3

#parsing date string
for i in range(len(matches_wip)):
  year = month = matches_wip.date[i].split('-')[0]
  month = matches_wip.date[i].split('-')[1]
  day = matches_wip.date[i].split('-')[2]
  date_obj = month+"/"+day+"/"+year
  matches_wip.date[i] = date_obj

matches_wip = matches_wip.sort_values(["date"], ascending = True).reset_index(drop=True)

In [ ]:
matches_wip.tail()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
31,1216526,2020,Abu Dhabi,10/16/2020,MI,KKR,KKR,bat,normal,0,MI,0,8,Q de Kock,"Sheikh Zayed Stadium, Abu Dhabi",Chris Gaffaney,Virender Sharma,Sundaram Ravi
32,1216522,2020,Dubai (DSC),10/17/2020,RR,RCB,RR,bat,normal,0,RCB,0,7,AB de Villiers,Dubai International Cricket Stadium,Anil Chaudhary,Nitin Menon,Paul Reiffel
33,1216509,2020,Sharjah,10/17/2020,DC,CSK,CSK,bat,normal,0,DC,0,5,S Dhawan,Sharjah Cricket Stadium,KN Ananthapadmanabhan,Richard Illingworth,Chettithody Shamshuddin
34,1216512,2020,Abu Dhabi,10/18/2020,SRH,KKR,SRH,field,tie,0,KKR,0,0,L Ferguson,"Sheikh Zayed Stadium, Abu Dhabi",Pashchim Pathak,Sundaram Ravi,Virender Sharma
35,1216517,2020,Dubai (DSC),10/18/2020,MI,KXIP,MI,bat,tie,0,KXIP,0,0,KL Rahul,Dubai International Cricket Stadium,Nitin Menon,Paul Reiffel,Anil Chaudhary
